## KPI
Realizaremos la creacion de los dataframes necesarios para el calculo de cada KPI a partir del dataframe original dk_siniestros


In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Importamos los datos

In [2]:
df_siniestros = pd.read_csv("Datasets\df_siniestros.csv")

## KPI 1
### Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

Creamos una nueva variable "Semestre" que basicamente divide cada año en dos partes

In [10]:
df_siniestros["Semestre"] = df_siniestros["Mes"].apply(lambda x: 1 if x <= 6 else 2)

Una vez que tenemos la columna Semestres realizamos la agrupacion por Año y Semestre para poder sumar la cantidad de homicidios que corresponde a cada uno

In [11]:
df_agrupado_semestral = df_siniestros.groupby(["Año","Semestre"]).size().reset_index(name="Homicidios")
df_agrupado_semestral

,Año,Semestre,Homicidios
0,2016,1,64
1,2016,2,78
2,2017,1,66
3,2017,2,71
4,2018,1,69
5,2018,2,79
6,2019,1,57
7,2019,2,47
8,2020,1,30
9,2020,2,50


Calculamos una nueva variable Homicidios Semestre Anterior donde se calcula para cada semestre anterior la cantidad de homicidios

In [12]:
df_agrupado_semestral["SemestreAnterior"] = df_agrupado_semestral["Homicidios"].shift(periods=1, fill_value=0)
df_agrupado_semestral

,Año,Semestre,Homicidios,SemestreAnterior
0,2016,1,64,0
1,2016,2,78,64
2,2017,1,66,78
3,2017,2,71,66
4,2018,1,69,71
5,2018,2,79,69
6,2019,1,57,79
7,2019,2,47,57
8,2020,1,30,47
9,2020,2,50,30


Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales / Población total) * 100,000 <br>

Calculamos la tasa de homicidios para cada semestre cada 100 000 habitantes, teniendo en cuenta que la poblacion de Buenos Aires al censo 2022 es de 3 120 612


In [13]:
df_agrupado_semestral["TasaHomicidios"] = (df_agrupado_semestral["Homicidios"]/3120612) * 100000
df_agrupado_semestral["TasaSemestreAnterior"] = (df_agrupado_semestral["SemestreAnterior"]/3120612) * 100000
df_agrupado_semestral

,Año,Semestre,Homicidios,SemestreAnterior,TasaHomicidios,TasaSemestreAnterior
0,2016,1,64,0,2.050880,0.000000
1,2016,2,78,64,2.499510,2.050880
2,2017,1,66,78,2.114970,2.499510
3,2017,2,71,66,2.275195,2.114970
4,2018,1,69,71,2.211105,2.275195
5,2018,2,79,69,2.531555,2.211105
6,2019,1,57,79,1.826565,2.531555
7,2019,2,47,57,1.506115,1.826565
8,2020,1,30,47,0.961350,1.506115
9,2020,2,50,30,1.602250,0.961350


Creamos una nueva variable: "Diferencia Porcentual"

In [14]:
df_agrupado_semestral["Diferencia Porcentual"] = (df_agrupado_semestral["TasaHomicidios"] - df_agrupado_semestral["TasaSemestreAnterior"]) / df_agrupado_semestral["TasaSemestreAnterior"] * 100
df_agrupado_semestral

,Año,Semestre,Homicidios,SemestreAnterior,TasaHomicidios,TasaSemestreAnterior,Diferencia Porcentual
0,2016,1,64,0,2.050880,0.000000,inf
1,2016,2,78,64,2.499510,2.050880,21.875000
2,2017,1,66,78,2.114970,2.499510,-15.384615
3,2017,2,71,66,2.275195,2.114970,7.575758
4,2018,1,69,71,2.211105,2.275195,-2.816901
5,2018,2,79,69,2.531555,2.211105,14.492754
6,2019,1,57,79,1.826565,2.531555,-27.848101
7,2019,2,47,57,1.506115,1.826565,-17.543860
8,2020,1,30,47,0.961350,1.506115,-36.170213
9,2020,2,50,30,1.602250,0.961350,66.666667


Guardamos el nuevo dataframe del KPI 1

In [15]:
df_agrupado_semestral_csv = "Datasets/df_kpi1.csv"
df_agrupado_semestral.to_csv(df_agrupado_semestral_csv, index=False)

## KPI 2 

### Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

Filtramos y seleccionamos aquellos registros donde la victima es motociclista

In [17]:
df_kpi2 = df_siniestros[(df_siniestros["Victima"] == "MOTO")]
df_kpi2.head(5)

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,...,Latitud,Participantes,Victima,Acusado,DiaSemana,MomentoDia,Rol,Sexo,Edad,Semestre
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,-34.687570,MOTO-AUTO,MOTO,AUTO,Viernes,Madrugada,CONDUCTOR,MASCULINO,19.0,1
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,-34.631894,MOTO-AUTO,MOTO,AUTO,Domingo,Mañana,CONDUCTOR,MASCULINO,30.0,1
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,-34.680930,MOTO-SD,MOTO,OTRO,Domingo,Madrugada,CONDUCTOR,MASCULINO,18.0,1
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves,Madrugada,CONDUCTOR,MASCULINO,29.0,1
5,2016-0008,1,2016-01-24,2016,1,24,18:30:00,18,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,...,-34.684759,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,Domingo,Noche,CONDUCTOR,MASCULINO,30.0,1


Agrupamos por Año y contamos la cantidad de homicidios

In [18]:
df_kpi2 = df_kpi2.groupby(["Año"]).size().reset_index(name="Homicidios")
df_kpi2

,Año,Homicidios
0,2016,63
1,2017,54
2,2018,57
3,2019,50
4,2020,27
5,2021,45


Creacion de la nueva variable Año anterior donde se indica la cantidad de homicidios ocurridos el año anterior 

In [20]:
df_kpi2["Año_Anterior"] = df_kpi2["Homicidios"].shift(periods=1, fill_value=0)
df_kpi2

,Año,Homicidios,Año_Anterior
0,2016,63,0
1,2017,54,63
2,2018,57,54
3,2019,50,57
4,2020,27,50
5,2021,45,27


Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100<br>

Obtenemos la tasa de homicidios cada 100000 habitantes teniendo en cuenta que la poblacion de Buenos Aires al censo 2022 es de 3 120 612

In [21]:
df_kpi2["Tasa_Homicidios"] = (df_kpi2["Homicidios"] / 3120612) * 100000
df_kpi2["Tasa_Año_Anterior"] = (df_kpi2["Año_Anterior"] / 3120612) * 100000
df_kpi2

,Año,Homicidios,Año_Anterior,Tasa_Homicidios,Tasa_Año_Anterior
0,2016,63,0,2.018835,0.000000
1,2017,54,63,1.730430,2.018835
2,2018,57,54,1.826565,1.730430
3,2019,50,57,1.602250,1.826565
4,2020,27,50,0.865215,1.602250
5,2021,45,27,1.442025,0.865215


Creacion de la variable Variacion Porcentual

In [22]:
df_kpi2["Variacion"] = ((df_kpi2["Tasa_Homicidios"] - df_kpi2["Tasa_Año_Anterior"]) / df_kpi2["Tasa_Año_Anterior"]) * 100
df_kpi2["Variacion"] = df_kpi2["Variacion"].replace([np.inf, -np.inf], np.nan)
df_kpi2

,Año,Homicidios,Año_Anterior,Tasa_Homicidios,Tasa_Año_Anterior,Variacion
0,2016,63,0,2.018835,0.000000,NaN
1,2017,54,63,1.730430,2.018835,-14.285714
2,2018,57,54,1.826565,1.730430,5.555556
3,2019,50,57,1.602250,1.826565,-12.280702
4,2020,27,50,0.865215,1.602250,-46.000000
5,2021,45,27,1.442025,0.865215,66.666667


Guardamos el nuevo dataframe del KPI 2

In [23]:
df_kpi2_csv = "Datasets/df_kpi2.csv"
df_kpi2.to_csv(df_kpi2_csv, index=False)